<img src="https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **SQL para Análise de Dados**
## **Módulo 36** | SQL: Projeto

Professora [Mariane Neiva](https://www.linkedin.com/in/mariane-neiva/)<br>
Aluno [Petrucio Willian](https://www.linkedin.com/in/petruciowillian/)<br>
kaggle https://www.kaggle.com/code/petruciowilliangomes/projetosql


---

# **Índice**

# Exploração e análise de dados de crédito com SQL
Esse notebook faz parte do curso SQL para análise de dados da EBAC.

Os dados:
Os dados representam informações de clientes de um banco e contam com as seguintes colunas:

idade = idade do cliente<br>
sexo = sexo do cliente (F ou M)<br>
dependentes = número de dependentes do cliente<br>
escolaridade = nível de escolaridade do clientes<br>
salario_anual = faixa salarial do cliente<br>
tipo_cartao = tipo de cartao do cliente<br>
qtd_produtos = quantidade de produtos comprados nos últimos 12 meses<br>
iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses<br>
meses_inativo_12m = quantidade de meses que o cliente ficou inativo<br>
limite_credito = limite de credito do cliente<br>
valor_transacoes_12m = valor das transações dos ultimos 12 meses<br>
qtd_transacoes_12m = quantidade de transacoes dos ultimos 12 meses<br>
A tabela foi criada no AWS Athena junto com o S3 Bucket com uma versão dos<br> dados disponibilizados em: https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset

#  Conjunto de dados

O [conjunto de dados](https://raw.githubusercontent.com/rhatiro/exercicios-SQL-para-Analise-de-Dados-EBAC/main/Mo%CC%81dulo%208%20-%20Projeto%20Final/database/credito8.csv) utilizado neste projeto consiste em uma subamostra do [dataset original](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/credito.csv) do professor [André Perez](https://www.linkedin.com/in/andremarcosperez/). É importante destacar que pode haver discrepâncias nas proporções das categorias nesta amostra parcial em relação ao conjunto de dados original. Essa subamostra foi extraída com o objetivo de facilitar o processamento dos dados para o projeto em questão. Para acessar o dataset completo, você pode visitar o repositório GitHub do professor [André Perez](https://github.com/andre-marcos-perez), disponível através do seguinte link: https://github.com/andre-marcos-perez/ebac-course-utils/blob/main/dataset/credito.csv

É válido ressaltar que as estimativas baseadas nesse conjunto de dados podem apresentar menor precisão em comparação com a amostra completa. Portanto, os resultados aqui apresentados não possuem a mesma fidelidade de uma análise realizada com o dataset completo. Eles são utilizados apenas para fins didáticos do curso e para demonstração pessoal no portfólio. No entanto, todas as *queries* podem ser igualmente aplicadas ao conjunto de dados original para obter conclusões confiáveis e úteis em análises mais aprofundadas.

Os dados fornecem informações sobre clientes de um banco e incluem as seguintes variáveis:

A tabela e as *queries* em linguagem SQL foram criadas através do [Amazon Athena](https://aws.amazon.com/pt/athena/) junto com o [Amazon Simple Storage Service (Amazon S3)](https://aws.amazon.com/pt/s3/).

[**Amazon Web Services**](https://aws.amazon.com/pt/), também conhecido como [**AWS**](https://aws.amazon.com/pt/), é uma plataforma de serviços de computação em nuvem, que formam uma plataforma de computação na nuvem oferecida pela Amazon.com.

[Voltar ao índice](#Índice)

In [6]:
# Import da biblioteca pandas para visualização dos arquivos CSV resultantes das queries

import pandas as pd


# **1. Criação da tabela**

A primeira etapa é a criação da tabela com o *dataset* armazenado no **Amazon S3 Bucket** por meio de uma *query* no **AWS Athena**, para posteriormente realizar as consultas.

```sql
CREATE EXTERNAL TABLE IF NOT EXISTS default.credito (
	`idade` int,
	`sexo` string,
	`dependentes` int,
	`escolaridade` string,
	`estado_civil` string,
	`salario_anual` string,
	`tipo_cartao` string,
	`qtd_produtos` bigint,
	`iteracoes_12m` int,
	`meses_inativo_12m` int,
	`limite_credito` float,
	`valor_transacoes_12m` float,
	`qtd_transacoes_12m` int
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
	'serialization.format' = ',',
	'field.delim' = ','
)
LOCATION 's3://bucket-transacoes-ebac-projetofinal/'
TBLPROPERTIES ('has_encrypted_data' = 'false');
```

[Voltar ao índice](#Índice)

# **2. EDA (Exploratory Data Analysis)**

[Voltar ao índice](#Índice)

## 2.1 Preview da tabela

> **Query 1:**

```sql
SELECT * FROM "default"."credito" limit 10;
```

In [39]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_1.csv'
query_1 = pd.read_csv(url, sep=',')

query_1.to_csv('./query_1.csv', sep=',', index=False)

query_1.head()


,idade,sexo,dependentes,escolaridade,estado_civil,salario_anual,tipo_cartao,qtd_produtos,iteracoes_12m,meses_inativo_12m,limite_credito,valor_transacoes_12m,qtd_transacoes_12m
0,45,M,3,ensino medio,casado,$60K - $80K,blue,5,3,1,12691.51,1144.90,42
1,49,F,5,mestrado,solteiro,menos que $40K,blue,6,2,1,8256.96,1291.45,33
2,51,M,3,mestrado,casado,$80K - $120K,blue,4,0,1,3418.56,1887.72,20
3,40,F,4,ensino medio,na,menos que $40K,blue,3,1,4,3313.03,1171.56,20
4,40,M,3,sem educacao formal,casado,$60K - $80K,blue,5,0,1,4716.22,816.08,28


Podemos analisar os dados em busca de valores nulos (NA) e ver quantos existem em cada coluna da tabela. Isso ajuda a entender onde faltam informações, o que é útil para planejar como lidar com esses valores ausentes, seja preenchendo, removendo ou tratando de outra forma.

[Voltar ao índice](#Índice)

## 2.2 Quantidade de linhas no dataset

> **Query 2:**

```sql
SELECT COUNT(*) AS quantidade_linhas
FROM credito;
```

In [25]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_2.csv'
query_2 = pd.read_csv(url, sep=',')

query_2.to_csv('./query_2.csv', sep=',', index=False)

query_2.head()

,quantidade_linhas
0,2564


O conjunto de dados completo possui 10.127 linhas, enquanto esta amostra contém 2.564 linhas, representando aproximadamente 25% do dataset original.

[Voltar ao índice](#Índice)

## 2.3 Tipos de dados em cada coluna

> **Query 3:**

```sql
DESCRIBE credito;
```

![query03](https://raw.githubusercontent.com/rhatiro/exercicios-SQL-para-Analise-de-Dados-EBAC/main/Mo%CC%81dulo%208%20-%20Projeto%20Final/img/query03.png)

> Os tipos dos dados no *dataset* podem ser descritos como:
> - `int` representa dados do tipo inteiro (numérico);
> - `string` representa dados do tipo texto ou cadeia de caracteres;
> - `bigint` representa dados do tipo inteiro longo (um número inteiro maior do que o tipo int);
> - `float` representa dados do tipo número de ponto flutuante (um número com parte fracionária).

[Voltar ao índice](#Índice)

## 2.4.1 Categorias da variável `escolaridade`

> **Query 4:**

```sql
SELECT DISTINCT escolaridade
FROM credito;
```

In [27]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_4.csv'
query_4 = pd.read_csv(url, sep=',')

query_4.to_csv('./query_4.csv', sep=',', index=False)

query_4.head()

,escolaridade
0,na
1,sem educacao formal
2,mestrado
3,graduacao
4,doutorado


> Podemos notar que há valores **nulos/ausentes** na variável categórica `escolaridade`, representados pelo valor **`na`**.

[Voltar ao índice](#Índice)

## 2.4.2 Categorias da variável `estado_civil`

> **Query 5:**

```sql
SELECT DISTINCT estado_civil
FROM credito;
```

In [28]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_5.csv'
query_5 = pd.read_csv(url, sep=',')

query_5.to_csv('./query_5.csv', sep=',', index=False)

query_5.head()

,estado_civil
0,na
1,divorciado
2,solteiro
3,casado


> Também podemos notar a presença de valores nulos na variável categórica `estado_civil`.

[Voltar ao índice](#Índice)

## 2.4.3 Categorias da variável `sexo`

> **Query 6:**

```sql
SELECT DISTINCT sexo
FROM credito;
```

In [29]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_6.csv'
query_6 = pd.read_csv(url, sep=',')

query_6.to_csv('./query_6.csv', sep=',', index=False)

query_6.head()

,sexo
0,F
1,M


> Diferente das demais colunas, não há valores nulos na variável categórica `sexo`.

[Voltar ao índice](#Índice)

## 2.5.1 Quantidade de clientes em cada faixa salarial

> **Query 7:**

```sql
SELECT salario_anual,
	COUNT(*) AS quantidade
FROM credito
GROUP BY salario_anual
ORDER BY CASE
		salario_anual
		WHEN 'menos que $40K' THEN 1
		WHEN '$40K - $60K' THEN 2
		WHEN '$60K - $80K' THEN 3
		WHEN '$80K - $120K' THEN 4
		WHEN '$120K +' THEN 5 ELSE 6
	END;
```

In [30]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_7.csv'
query_7 = pd.read_csv(url, sep=',')

query_7.to_csv('./query_7.csv', sep=',', index=False)

query_7.head()

,salario_anual,quantidade
0,menos que $40K,701
1,$40K - $60K,467
2,$60K - $80K,451
3,$80K - $120K,488
4,$120K +,222


![query07.png](https://raw.githubusercontent.com/rhatiro/exercicios-SQL-para-Analise-de-Dados-EBAC/main/Mo%CC%81dulo%208%20-%20Projeto%20Final/img/query07.png)

> Ao desconsiderar a presença dos 235 valores nulos na variável `salario_anual`, podemos observar que a maioria dos clientes recebem anualmente a quantia abaixo de \\$40.000,00 e a minoria acima de \\$120.000,00.

[Voltar ao índice](#Índice)

## 2.5.2 Quantidade de clientes de acordo com cada tipo de cartão

> **Query 8:**

```sql
SELECT tipo_cartao,
	COUNT(*) AS quantidade
FROM credito
GROUP BY tipo_cartao
ORDER BY quantidade DESC;
```

In [31]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_8.csv'
query_8 = pd.read_csv(url, sep=',')

query_8.to_csv('./query_8.csv', sep=',', index=False)

query_8.head()

,tipo_cartao,quantidade
0,blue,2453
1,silver,93
2,gold,16
3,platinum,2


![query08.png](https://raw.githubusercontent.com/rhatiro/exercicios-SQL-para-Analise-de-Dados-EBAC/main/Mo%CC%81dulo%208%20-%20Projeto%20Final/img/query08.png)

> A maioria possui o tipo de cartão *blue*, dando a entender que supostamente façam parte dos clientes com faixa de salário anual de menor valor.

[Voltar ao índice](#Índice)

## 2.5.3 Quantidade de clientes de acordo com o sexo

> **Query 9:**

```sql
SELECT sexo,
	COUNT(*) AS quantidade
FROM credito
GROUP BY sexo
ORDER BY quantidade DESC;
```

In [32]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_9.csv'
query_9 = pd.read_csv(url, sep=',')

query_9.to_csv('./query_9.csv', sep=',', index=False)

query_9.head()

,sexo,quantidade
0,M,1563
1,F,1001


![query09.png](https://raw.githubusercontent.com/rhatiro/exercicios-SQL-para-Analise-de-Dados-EBAC/main/Mo%CC%81dulo%208%20-%20Projeto%20Final/img/query09.png)

> Neste conjunto de dados, a proporção de clientes do sexo masculino e feminino não está equilibrada. Portanto é algo que deve ser levado em consideração ao relacionar esta variável com as demais.

[Voltar ao índice](#Índice)

## 2.6.1 Análise da idade dos clientes (média, mínima e máxima)

> **Query 10:**

```sql
SELECT ROUND(AVG(idade)) AS idade_media,
	MIN(idade) AS idade_minima,
	MAX(idade) AS idade_maxima
FROM credito;
```

In [33]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_10.csv'
query_10 = pd.read_csv(url, sep=',')

query_10.to_csv('./query_10.csv', sep=',', index=False)

query_10.head()

,idade_media,idade_minima,idade_maxima
0,46.0,26,73


>Idade Média (idade_media): O valor é 46 anos, indicando que a média das idades na amostra é de 46 anos. Este valor sugere que a maioria dos indivíduos possui uma idade em torno dessa faixa.

## 2.6.2 Análise dos gastos dos clientes (média, mínima e máxima)

> **Query 11:**

```sql
SELECT ROUND(AVG(valor_transacoes_12m), 2) AS valor_medio_transacoes,
	MIN(valor_transacoes_12m) AS valor_minimo_transacoes,
	MAX(valor_transacoes_12m) AS valor_maximo_transacoes
FROM credito;
```

In [34]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_11.csv'
query_11 = pd.read_csv(url, sep=',')

query_11.to_csv('./query_11.csv', sep=',', index=False)

query_11.head()

,valor_medio_transacoes,valor_minimo_transacoes,valor_maximo_transacoes
0,1820.33,510.16,4776.58


>O valor médio das transações é de 1820,33. Isso indica que, em média, cada transação realizada tem um valor em torno de 1820,33. Este valor médio ajuda a entender o comportamento geral dos gastos ou valores transacionados pelos indivíduos.

[Voltar ao índice](#Índice)

## 2.7.1 Limite mínimo de crédito de acordo com o tipo de cartão, sexo e escolaridade

> **Query 12:**

```sql
SELECT COUNT(*) as quantidade,
	MIN(limite_credito) AS limite_credito_maximo,
	tipo_cartao,
	sexo,
	escolaridade
FROM credito
WHERE escolaridade != 'na'
GROUP BY sexo,
	escolaridade,
	tipo_cartao
ORDER BY limite_credito_maximo ASC;
```

In [35]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_12.csv'
query_12 = pd.read_csv(url, sep=',')

query_12.to_csv('./query_12.csv', sep=',', index=False)

query_12.head()

,quantidade,limite_credito_maximo,tipo_cartao,sexo,escolaridade
0,91,1438.03,blue,F,graduacao
1,92,1438.04,blue,F,doutorado
2,133,1438.07,blue,M,doutorado
3,316,1438.08,blue,F,mestrado
4,148,1438.12,blue,F,sem educacao formal


[Voltar ao índice](#Índice)

## 2.7.2 Limite máximo de crédito de acordo com o tipo de cartão, sexo e escolaridade

> **Query 13:**

```sql
SELECT COUNT(*) as quantidade,
	MAX(limite_credito) AS limite_credito_maximo,
	tipo_cartao,
	sexo,
	escolaridade
FROM credito
WHERE escolaridade != 'na'
GROUP BY sexo,
	escolaridade,
	tipo_cartao
ORDER BY limite_credito_maximo DESC;
```

In [36]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_13.csv'
query_13 = pd.read_csv(url, sep=',')

query_13.to_csv('./query_13.csv', sep=',', index=False)

query_13.head()

,quantidade,limite_credito_maximo,tipo_cartao,sexo,escolaridade
0,203,34516.99,blue,M,sem educacao formal
1,2,34516.98,gold,M,mestrado
2,462,34516.97,blue,M,mestrado
3,1,34516.96,platinum,F,doutorado
4,7,34516.96,silver,M,sem educacao formal


[Voltar ao índice](#Índice)

É possível observar que não há uma relação considerável da variável `escolaridade` com o limite de crédito dos clientes. Os resultados dos limites mínimos de acordo com as separações categóricas demonstram que o tipo de cartão *blue* representa a maior parte dos limites mais baixos enquanto os demais representam os mais altos.

A variável `sexo` demonstra que clientes do sexo masculino possuem limites máximos de crédito mais altos enquanto clientes do sexo feminino possuem limites máximos de crédito mais baixos.

## 2.8 Análise da quantidade e valores das transações de acordo com o sexo e estado civil

> **Query 14:**

```sql
SELECT COUNT(*) AS quantidade,
	sexo,
	estado_civil,
	MIN(valor_transacoes_12m) AS valor_minimo_transacoes,
	MAX(valor_transacoes_12m) AS valor_maximo_transacoes,
	AVG(valor_transacoes_12m) AS valor_medio_transacoes
FROM credito
WHERE estado_civil != 'na'
GROUP BY sexo,
	estado_civil
ORDER BY quantidade ASC;
```

In [37]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_14.csv'
query_14 = pd.read_csv(url, sep=',')

query_14.to_csv('./query_14.csv', sep=',', index=False)

query_14.head()

,quantidade,sexo,estado_civil,valor_minimo_transacoes,valor_maximo_transacoes,valor_medio_transacoes
0,69,F,divorciado,1039.44,4370.80,2024.6688
1,92,M,divorciado,781.27,4605.89,1959.7296
2,308,F,solteiro,777.75,4776.58,2035.2687
3,458,M,solteiro,530.36,4686.93,1978.7189
4,591,F,casado,510.16,4740.94,1707.7339


> Nesta análise podemos notar que a quantidade de transações tem uma relação considerável com a variável `estado_civil`, sendo que a maior parte foi efetuada por clientes casados e a menor parte por clientes divorciados. Em contrapartida, a média de valor das transações também se diferencia de acordo com o estado civil mas de forma contrária, na qual a média mais alta são dos clientes divorciados e a média mais baixa são dos clientes casados, assim como pode ser constatado na relação dos valores mínimos de transações, dando a entender que a baixa quantidade de transações possívelmente tenha relação com o alto valor das mesmas.
>
> Em relação ao sexo do cliente, a variação não chega a ser tão grande, havendo somente uma leve discrepância entre os valores mínimos das transações, demonstrando ser mais alto entre clientes do sexo feminino que estão divorciadas. Já entre as categorias da variável `estado_civil`, observa-se que clientes solteiros e divorciados do sexo masculino efetuaram uma quantidade maior transações do que clientes do sexo feminino, porém a quantidade é um pouco maior entre clientes casados.

[Voltar ao índice](#Índice)

## 2.9 Média da quantidade de transações, valor das transações e limite de crédito de acordo com o tipo de cartão e salário anual

> **Query 15:**

```sql
SELECT COUNT(*) AS quantidade,
	tipo_cartao,
	salario_anual,
	ROUND(AVG(qtd_transacoes_12m)) AS media_qtd_transacoes,
	ROUND(AVG(valor_transacoes_12m), 2) AS media_valor_transacoes,
	ROUND(AVG(limite_credito), 2) AS media_limite_credito
FROM credito
WHERE salario_anual != 'na'
GROUP BY tipo_cartao,
	salario_anual
ORDER BY CASE
		tipo_cartao
		WHEN 'blue' THEN 1
		WHEN 'silver' THEN 2
		WHEN 'gold' THEN 3 ELSE 4
	END,
	CASE
		salario_anual
		WHEN 'menos que $40K' THEN 1
		WHEN '$40K - $60K' THEN 2
		WHEN '$60K - $80K' THEN 3
		WHEN '$80K - $120K' THEN 4
		WHEN '$120K +' THEN 5 ELSE 6
	END;
```

In [38]:
# Use the 'raw' URL for the CSV file on GitHub
url = 'https://raw.githubusercontent.com/petwillian/modulo_36/refs/heads/main/query_15.csv'
query_15 = pd.read_csv(url, sep=',')

query_15.to_csv('./query_15.csv', sep=',', index=False)

query_15.head()

,quantidade,tipo_cartao,salario_anual,media_qtd_transacoes,media_valor_transacoes,media_limite_credito
0,677,blue,menos que $40K,43.0,1857.24,3797.09
1,449,blue,$40K - $60K,42.0,1841.51,4832.76
2,431,blue,$60K - $80K,41.0,1814.45,8061.38
3,461,blue,$80K - $120K,40.0,1748.57,13736.87
4,212,blue,$120K +,40.0,1707.02,17105.39


> Nesta análise podemos ver novamente a relação entre o limite de crédito com o tipo de cartão do cliente, sendo que as médias mais baixas de limite são dos clientes que possuem o tipo de cartão *blue*, aumentando de acordo com a faixa salarial anual. As demais médias de quantidade e valor das transações não possuem relação tão considerável com as variáveis deste resultado.

[Voltar ao índice](#Índice)

# **3. Conclusão:**

Essas foram algumas das análises extraídas do conjunto de dados, fornecendo insights sobre o perfil dos clientes e seus comportamentos de transação. Resumidamente, destacam-se os seguintes insights:
- A maioria dos clientes possui um salário anual abaixo de $40.000,00, apresentando o tipo de cartão *blue* e limites de crédito mais baixos. Isso indica que a campanha de marketing direcionada a esse grupo específico pode ter um impacto significativo, considerando sua representatividade na base de clientes.
- Ao analisar a variável sexo, observa-se que os clientes do sexo masculino possuem limites máximos de crédito mais elevados em comparação aos clientes do sexo feminino. No entanto, é importante ressaltar que a proporção de clientes masculinos e femininos na base de dados não está equilibrada, o que pode influenciar essa discrepância.
- Verificou-se que os clientes casados são responsáveis pela maior parte das transações, enquanto os clientes divorciados apresentam uma menor participação. Além disso, os clientes divorciados possuem uma média mais alta de valor das transações, sugerindo uma possível relação entre a menor quantidade de transações e seu valor mais elevado.
- Quando analisados por gênero, os clientes solteiros e divorciados do sexo masculino realizaram mais transações em comparação às clientes do sexo feminino. No entanto, a quantidade de transações entre clientes casados é ligeiramente superior. Esses dados destacam a importância de considerar o estado civil e o gênero ao desenvolver estratégias de marketing personalizadas.
- As médias de limite de crédito mais baixas foram encontradas entre os clientes com o tipo de cartão *blue*. Além disso, em cada categoria de tipo de cartão, a média de limite de crédito aumenta de acordo com a faixa salarial anual do cliente. Essa informação pode ser relevante para ajustar as ofertas e comunicações de acordo com o perfil de cada cliente, considerando sua capacidade de gastos.

Esses insights oferecem uma visão mais completa do comportamento dos clientes e podem servir como base para a tomada de decisões estratégicas, permitindo uma segmentação mais eficiente e personalizada dos esforços de marketing.

**Com base nesta análise geral do conjunto de dados, conclui-se que, para uma possível campanha de marketing direcionada aos clientes mais ativos, é recomendado um foco maior nos clientes casados que possuem o tipo de cartão *blue* e apresentam um salário anual abaixo de $40.000,00. Essa estratégia visa otimizar os esforços de marketing, concentrando-se nos segmentos de clientes com maior potencial de resposta e engajamento, permitindo uma alocação mais eficiente dos recursos disponíveis. Ao direcionar a campanha para essa demografia específica, espera-se aumentar a eficácia das ações de marketing e obter resultados mais favoráveis em termos de aquisição e retenção de clientes.**

[Voltar ao índice](#Índice)

---